# Librairie chez Lapage

>  Partie 3 _ Tests statistiques



## Librairies & df_final

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sb
import seaborn as sns
import datetime
from datetime import datetime
from datetime import date
import scipy.stats as st

In [ ]:
def matrix(df):
    pd.plotting.scatter_matrix((df), alpha=0.2)
    return matrix

In [ ]:
def histogram(df):
    return df.hist(figsize=(20,8))

In [ ]:
def eta2(df, quant, qual):
    # on créé un array qui contient les valeurs de chaques classes séparé dans des sous array
    class_valeurs = []
    for c in  df[quant].unique():
        class_valeurs.append(df.loc[df[quant]==c, qual].values)
    
    # on calcule eta2
    y =  np.concatenate(class_valeurs)
    moyenne_y = y.mean()

    class_info = []
    for y_i in class_valeurs:
        class_info.append({
            'n_i': len(y_i),
            'moyenne_i': y_i.mean()
        })
        
    SCE = sum([ c['n_i'] * (c['moyenne_i'] - moyenne_y)**2 for c in class_info])
    SCT = sum([(y_ij - moyenne_y)**2 for y_ij in y])
    
    return SCE / SCT

In [ ]:
df_final = pd.read_csv(r'C:\Users\melle\FORMATION\P6_Mendy\P6_DF_FINAL.csv')
df_final = df_final[['client_id', 'birth', 'sex', 'age', 'id_prod', 'price',
       'categ', 'session_id', 'date_longue', 'date_courte',
       'date_année']]
df_final.head(2)

In [ ]:
df_final.dtypes

In [ ]:
temps = [ 'date_longue' ,'date_courte', 'date_année']
for var in temps:
    df_final[var] = df_final[var].astype("datetime64[ns]")

In [ ]:
# google collab
# Création calendrier par semaine
df_final.date_longue.dt.isocalendar().week
df_final["num_semaine"] = df_final.date_longue.dt.isocalendar().week

In [ ]:
client = [ 'birth' ,'age', 'categ',"num_semaine"]
for var in client:
    df_final[var] = df_final[var].astype(int)

In [ ]:
df_final["type_client"]=1
labels = ["BtoC","BtoB" ]


In [ ]:
df_final.type_client = df_final.type_client.replace(to_replace=1, value="B2C", regex=True)

In [ ]:
df_final.loc[(df_final.client_id == "c_1609"),'type_client']='B2B'
df_final.loc[(df_final.client_id == "c_4958"),'type_client']='B2B'
df_final.loc[(df_final.client_id == "c_6714"),'type_client']='B2B'
df_final.loc[(df_final.client_id == "c_3454"),'type_client']='B2B'

In [ ]:
df_final["génération"]=1
bins = [0,30,50,100]
labels = "Moins de 30","31-50","51","et_plus" 
df_final["génération"]=pd.cut(df_final["age"], bins, labels)
df_final.isna().sum()

In [ ]:
df_final.sample(10)

In [ ]:
df_final.price.min()

# Profil Report

In [ ]:
BtoB = ["c_1609", "c_4958", "c_6714", "c_3454"]
BtoB = df_final.loc[df_final["client_id"].isin(BtoB),:]
BtoB = BtoB[['client_id', 'birth', 'sex', 'age', 'id_prod', 'price',
       'categ', 'session_id', 'date_longue', 'date_courte',
       'date_année', 'génération']]


#B2C
clé_filtre = BtoB["client_id"].values.tolist()
BtoC = df_final[~df_final.client_id.isin(clé_filtre)]
BtoC = BtoC[['client_id', 'birth', 'sex', 'age', 'id_prod', 'price',
       'categ', 'session_id', 'date_longue', 'date_courte',
       'date_année', 'génération']]

In [ ]:
BtoB.client_id.unique()

# Tests statistiques

In [ ]:
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import pearsonr
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy.stats import anderson
from scipy.stats import spearmanr


In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols

import pandas.testing as tm

In [ ]:
# Visualisation
from pandas.plotting import scatter_matrix
from pandas.plotting import radviz


## Visualisation Dataframe

# Genre _ Categ

In [ ]:
Genre_categ = df_final.groupby(["categ","sex"]).agg({"price":"sum", "id_prod":"count","sex":"count"})
Genre_categ.rename(columns = {'id_prod': 'nombre_de_vente'}, inplace=True)
Genre_categ.rename(columns = {'sex': 'Nombre_de_personnes'}, inplace=True)
Genre_categ.rename(columns = {'price': 'CA'}, inplace=True)
Genre_categ["CA"] = Genre_categ["CA"].round(2)
Genre_categ = Genre_categ.reset_index()
Genre_categ




In [ ]:
X = "sex"
Y = "categ"

cont = df_final[[X,Y]].pivot_table(index=X,columns=Y,aggfunc=len,margins=True)
cont

In [ ]:
Genre_categ1 = pd.pivot_table(Genre_categ, values=['CA', 'nombre_de_vente', 
                                                  'Nombre_de_personnes'], 
                             index=['sex','categ'], aggfunc=np.sum)
Genre_categ1.T


In [ ]:

var = 'CA', 'nombre_de_vente', 'Nombre_de_personnes'
for i in var:
    Genre_categ[i] = (Genre_categ[i]/Genre_categ[i].sum())*100
print(Genre_categ)
    

In [ ]:
Genre_categ.rename(columns = {'nombre_de_vente': '%ventes'}, inplace=True)
Genre_categ.rename(columns = {'Nombre_de_personnes': '%clients'}, inplace=True)
Genre_categ.rename(columns = {'CA': '%CA'}, inplace=True)

In [ ]:
Genre_categ.round(2)

In [ ]:
Genre_categ.reset_index(inplace=True)
Genre_categ

In [ ]:
Genre_categ.columns

In [ ]:
Genre_categ = pd.pivot_table(Genre_categ, values=['%CA', '%ventes', '%clients'], 
                             index=['sex'], columns=['categ'], aggfunc=np.sum)
Genre_categ

In [ ]:
Genre_categ = Genre_categ.T
Genre_categ.reset_index(inplace=True)
Genre_categ.round(2)

In [ ]:
Genre_categ = Genre_categ.set_index("categ","sex")
Genre_categ

## Visualisation


In [ ]:
sns.set()
sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="categ", y="price", hue="sex", data=df_final)
plt.xlabel('Categ')
plt.ylabel('Price')
plt.title('Boîtes à moustaches_ categ, price, sex')

plt.show()

print("La Mediane est la meme, sauf en categ 2, mediane identique mais variance plus grance chez les hommes")

In [ ]:
g = sns.catplot(x="age", y="price",
                hue="sex", col="categ",
                data=df_final, kind="bar",
                height=4, aspect=.7);

In [ ]:
cont

In [ ]:
# Tableau de contingence

tx = cont.loc[:,["All"]]
ty = cont.loc[["All"],:]
n = len(df_final)
indep = (tx.dot(ty) / n)

c = cont.fillna(0) # On remplace les valeurs nulles par 0
measure = (c-indep)**2/indep
xi_n = measure.sum().sum().round(2)
table = measure/xi_n.round(2)
sns.heatmap(table.iloc[:-1,:-1],annot=c.iloc[:-1,:-1])
plt.show()

## Test de Khi2

In [ ]:
corr_sex_categ =  pd.crosstab(df_final["sex"], df_final.categ, normalize = True)
corr_sex_categ

In [ ]:
sns.heatmap(corr_sex_categ)

In [ ]:
# kHI2 SUR CONT

In [ ]:
cont

In [ ]:
from scipy.stats import chi2_contingency
chi2, pvalue, degrees, expected = chi2_contingency(cont)
print("Khi2: {}".format(chi2) , "Degrees: {}".format(degrees), "P-value: {}".format(pvalue))
p_value = pvalue
# si pvalue inferieur a .05 alors il y a pas de lien, on rejette donc pas H0

if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

In [ ]:
from scipy.stats import chi2_contingency
chi2, pvalue, degrees, expected = chi2_contingency(corr_sex_categ)
print("Khi2: {}".format(chi2) , "Degrees: {}".format(degrees), "P-value: {}".format(pvalue))

In [ ]:
p_value = pvalue
# si pvalue inferieur a .05 alors il y a pas de lien, on rejette donc pas H0

if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

## Anova: En considerant la variable categ comme quantitative on peux effectuer un test one way Anova

In [ ]:
model = ols('categ ~ sex',
                data=df_final).fit()
                
anova = sm.stats.anova_lm(model, typ=2)
anova

In [ ]:
p_value = anova["PR(>F)"]["sex"]

In [ ]:
p_value = pvalue
# si pvalue inferieur a .05 alors il y a pas de lien, on rejette donc pas H0

if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

In [ ]:
# Selon le sexe
stats.f_oneway(df_final['categ'][df_final['sex'] == 'f'],
               df_final['categ'][df_final['sex'] == 'm'])

In [ ]:
p_value = pvalue
# si pvalue inferieur a .05 alors il y a pas de lien, on rejette donc pas H0
if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

# le lien entre l’âge des clients et le montant total des achats, la fréquence d’achat, la taille du panier moyen et les catégories des livres achetés

## Observation des distributions

In [ ]:
print(df_final.describe( ).round(2))

In [ ]:
histogram(df_final);

In [ ]:
corr = df_final.corr(method="spearman", min_periods=3)

corr.style.background_gradient(cmap='coolwarm')

## Analyse des correlation

In [ ]:
corr = df_final.corr(method="spearman", min_periods=3)

corr.style.background_gradient(cmap='coolwarm')

In [ ]:
sns.heatmap(df_final.corr(), annot = True)

In [ ]:
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot

# Création des DF

## frequence_client

In [ ]:
frequence_client = df_final.sort_values(by=["client_id","date_longue"])
reference_value = frequence_client["date_longue"][1]
frequence_client['fréquence_d_achat'] = (
frequence_client['date_longue'] - frequence_client['date_longue'].shift(fill_value=reference_value))
frequence_client["fréquence_d_achat"] = frequence_client["fréquence_d_achat"].dt.days
frequence_client["fréquence_d_achat"] = frequence_client["fréquence_d_achat"].mask(frequence_client["fréquence_d_achat"]<=0,0)
# modification du 1er delai de 57 a 0
frequence_client.loc[:181733 , "fréquence_d_achat"]=0
frequence_client

## Age_CA

In [ ]:
Age_CA = frequence_client.groupby(by="age").agg({"price":"sum", "fréquence_d_achat":"mean","id_prod":"count"})
Age_CA = Age_CA.reset_index()
Age_CA.rename(columns = {'id_prod': 'nombre_de_vente'}, inplace=True)
Age_CA.rename(columns = {'price': 'CA'}, inplace=True)
Age_CA["CA"] = Age_CA["CA"].round(2)
Age_CA["fréquence_d_achat"] = Age_CA["fréquence_d_achat"].astype(int)
Age_CA.head()

In [ ]:
Age_CA.CA.isna().sum()

In [ ]:
Age_CA

## Age_categ

In [ ]:

#df avec categ
Age_categ = frequence_client.groupby(by=["age","categ"]).agg({"price":"sum", "fréquence_d_achat":"mean","id_prod":"count"})
Age_categ = Age_categ.reset_index()
Age_categ.rename(columns = {'id_prod': 'nombre_de_vente'}, inplace=True)
Age_categ.rename(columns = {'price': 'CA'}, inplace=True)
Age_categ["CA"] = Age_categ["CA"].round(2)
Age_categ["fréquence_d_achat"] = Age_categ["fréquence_d_achat"].astype(int)
Age_categ.head()

## génération_categ

In [ ]:
génération_categ = frequence_client.groupby(by=["génération","categ"]).agg({"price":"sum", "fréquence_d_achat":"mean","id_prod":"count"})
génération_categ = génération_categ.reset_index()
génération_categ.rename(columns = {'id_prod': 'nombre_de_vente'}, inplace=True)
génération_categ.rename(columns = {'price': 'CA'}, inplace=True)
génération_categ["CA"] = génération_categ["CA"].round(2)
génération_categ["fréquence_d_achat"] = génération_categ["fréquence_d_achat"].astype(int)
génération_categ.head()

## panier_moyen_age

In [ ]:
#df panier moyen
CA_par_session=  df_final.groupby(by=["age","session_id"]).agg({"price":"sum"})
CA_par_session = CA_par_session.reset_index() 

CA_par_session.head(20)

In [ ]:
print("Le panier moyen est de: {:.4}€".format(CA_par_session.price.mean()))

In [ ]:
panier_moyen_age.shape

In [ ]:
panier_moyen_age =  CA_par_session.groupby(by="age").agg({"price":"mean"})
panier_moyen_age.reset_index(inplace=True)
panier_moyen_age

In [ ]:
ax = sns.lmplot(x="age", y="price", data=panier_moyen_age, ci=None, line_kws={'color':'black'})
ax.set(xlabel='Age', ylabel='Panier Moyen')
plt.show()

## panier_moyen_age_categ

In [ ]:
CA_par_session_categ=  df_final.groupby(by=["age","session_id","categ"]).agg({"price":"sum"})
CA_par_session_categ = CA_par_session_categ.reset_index() 

CA_par_session_categ.head(20)

In [ ]:
panier_moyen_age_categ =  CA_par_session_categ.groupby(by=["age","categ"]).agg({"price":"mean"})
panier_moyen_age_categ.reset_index(inplace=True)
panier_moyen_age_categ

# df_test

## df_test_Age

In [ ]:
df_test_Age = pd.merge(Age_CA, panier_moyen_age, on=["age"])

df_test_Age.rename(columns = {'price': 'Panier_moyen'}, inplace=True)
df_test_Age["Panier_moyen"] = df_test_Age["Panier_moyen"].round(2)


In [ ]:
df_test_Age["génération"]=1
bins = [0,30,50,100]
labels = ["Moins de 30","31-50","51 et plus" ]
df_test_Age["génération"]=pd.cut(df_test_Age["age"], bins, labels)
df_test_Age

## df_test_age_categ

In [ ]:
df_test_age_categ = pd.merge(Age_categ, panier_moyen_age_categ, on=["age","categ"])
df_test_age_categ.rename(columns = {'price': 'Panier_moyen'}, inplace=True)
df_test_age_categ["Panier_moyen"] = df_test_age_categ["Panier_moyen"].round(2)

In [ ]:
df_test_age_categ["génération"]=1
bins = [0,30,50,100]
labels = ["Moins de 30","31-50","51 et plus" ]
df_test_age_categ["génération"]=pd.cut(df_test_age_categ["age"], bins, labels)
df_test_age_categ

In [ ]:

var = 'CA', 'fréquence_d_achat', 'nombre_de_vente','Panier_moyen'
for i in var:
    df_test_age_categ2[i] = (df_test_age_categ[i]/df_test_age_categ[i].sum())*100
print(df_test_age_categ2)
    

# Test 1 to 1


## Age _ CA

In [ ]:
ax = sns.lmplot(x="age", y="CA", data=df_test_Age, ci=None, line_kws={'color':'black'})
ax.set(xlabel='Age', ylabel='CA')

plt.show()

In [ ]:
df_test_Age.génération.unique()

In [ ]:
sns.set()
sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="CA", y="génération", data=df_test_Age)
plt.xlabel('CA')
plt.ylabel('Génération')
plt.title("""Boîtes à moustaches: CA selon l'age""")

plt.show()


In [ ]:
sns.set()
sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="categ", y="nombre_de_vente",hue="génération", data=df_test_age_categ)
plt.xlabel('Nombre de vente', size=18)
plt.ylabel('Génération', size=18)
plt.title("""Boîtes à moustaches: nombre_de_vente selon l'age""",size=20)

plt.show()


In [ ]:
data1 = df_test_Age["age"]
data2 = df_test_Age["CA"]
coef, p = spearmanr(data1, data2)
print('Coefficient de correlation methode Spearman  : %.3f' % coef, "p_value: {}".format(p))


if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")   


In [ ]:
data1 = df_test_Age["age"]
data2 = df_test_Age["CA"]
coef, p = pearsonr(data1, data2)
print(coef, p)
print('Coefficient de correlation methode Pearson  : %.3f' % coef, "p_value: {:.2}".format(p))


if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

## Age _ nombre_de_vente

In [ ]:
ax = sns.lmplot(x="age", y="nombre_de_vente", data=df_test_Age, ci=None, line_kws={'color':'black'})
ax.set(xlabel='Age', ylabel='nombre_de_vente')
plt.grid("visible")
plt.show()

In [ ]:
sns.set()
sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="nombre_de_vente", y="génération", data=df_test_Age)
plt.xlabel('Nb de ventes')
plt.ylabel('Génération')
plt.title("""Boîtes à moustaches: Nb de ventes selon l'age""")

plt.show()


In [ ]:
khi2_age_nombre_de_vente = pd.crosstab(df_test_Age["age"], df_test_Age.nombre_de_vente, normalize="index")


In [ ]:
data1 = df_test_Age["age"]
data2 = df_test_Age["nombre_de_vente"]
coef, p = pearsonr(data1, data2)
print(coef, p)
print('Coefficient de correlation methode Pearson  : %.3f' % coef, "p_value: {}".format(p))

if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

## Age _ fréquence_d_achat

In [ ]:
ax = sns.lmplot(x="age", y="fréquence_d_achat", data=df_test_Age, ci=None, line_kws={'color':'black'})
ax.set(xlabel='Age', ylabel='fréquence_d_achat')
plt.show()

In [ ]:
df_test_Age

In [ ]:
print('Coefficient de correlation methode Spearman  : %.3f' % coef, "p_value: {}".format(p))

data1 = df_test_Age["age"]
data2 = df_test_Age["fréquence_d_achat"]
coef, p = spearmanr(data1, data2)
print(coef, p)
if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

In [ ]:
print('Coefficient de correlation methode Pearson  : %.3f' % coef, "p_value: {}".format(p))

data1 = df_test_Age["age"]
data2 = df_test_Age["fréquence_d_achat"]
coef, p = pearsonr(data1, data2)
print(coef, p)
if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

In [ ]:
# ANOVA avec génération comme variable 

In [ ]:
alpha = 0.05
x = df_test_Age["age"].var()
y = df_test_Age["fréquence_d_achat"].var()
F = x/y
df1 = len(str(x)) - 1
df2 = len(str(y)) - 1
alpha = 0.05
p_value = st.f.cdf(F, df1, df2)
print(p_value)
if p_value > alpha:
    print("Nous rejetons l'hypothèse H0 var(x) = var(y), il y a ici p_value: {:.2%} de chance que H0 soit rejeté à tord ".format(p) )
else:
    print("var(x) = var(y), on ne peux rejeter l(hypothèse H0, il y a ici p_value: {:.2%} de chance que H0 ne soit pas rejeté à tord ".format(p) )

In [ ]:
reg_simp = smf.ols('fréquence_d_achat ~ génération ', data=df_test_Age).fit()

In [ ]:
print(reg_simp.summary())

In [ ]:
shapiro(reg_simp.resid)

In [ ]:
# Test la  normalité des résidus
shapiro(reg_simp.resid)
p=pvalue
if p < 0.05:
    print("Ici, l'hypothèse de normalité est remise en cause") 
else:
    print("Ici, l'hypothèse de normalité semble vérifiée")   
print(p)

In [ ]:
#Création categ pour Anova One-way
Moins_30 = df_test_age_categ.query("age<30")
Age_30_50 = df_test_age_categ.query("age>30 & age<50")
Age_50_et_plus = df_test_age_categ.query("age>50")

In [ ]:
f,p = st.f_oneway(Moins_30["fréquence_d_achat"],
                     Age_30_50["fréquence_d_achat"],
                     Age_50_et_plus["fréquence_d_achat"])
print(f'la f-valeur : {f:.2f} et la p-valeur : {p}.')

if p_value<0.05:
    print("P_value = {}".format(p_value), "La p-valeur de ce test est inférieur à 5%,  l\'hypothèse H0 peux etre rejetée, il existe ici pvalue:, {}".format(p_value), "de chance que H0 ne soit pas rejeté à tord (Beta: Erreur de 2nd espèce)""")
    
else: 
    print("P_value = {}".format(p_value),"La p-valeur est supérieure à 5%, cela signifie qu'il existe pas de lien entre les variables, ce qui incite à rejeter l'hypothèse  H0""")

In [ ]:
eta2(df_test_Age, 'fréquence_d_achat', 'age')

In [ ]:
plt.figure(figsize=(5,5))
plt.boxplot([Moins_30["fréquence_d_achat"],
             Age_30_50["fréquence_d_achat"],
            Age_50_et_plus["fréquence_d_achat"]
], vert=False, showmeans=True, showfliers=False)

plt.xlabel("fréquence d'achat (délai en jour)")
plt.yticks([1,2,3], ['Moins_30', 'Age_30_50', 'Age_50_et_plus'])
plt.show()

In [ ]:
eta2()

## Age _ Categ

In [ ]:
df_test_Age

In [ ]:
df_test_age_categ

In [ ]:
sns.set()
sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="categ", y="price", hue="génération", data=df_final)
plt.xlabel('Categ')
plt.ylabel('Price')
plt.title('Boîtes à moustaches_ categ, price, génération',size=20)

plt.show()


In [ ]:
df_test_age_categ.head(3)

In [ ]:
sns.set()
#sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="categ", y="CA", hue="génération", data=df_test_age_categ)
plt.xlabel('Categ')
plt.ylabel('CA')
plt.title('Boîtes à moustaches_ categ, CA par génération')

plt.show()


## Test de Bartlett _ Egalité des variances

In [ ]:
alpha = 0.05
x = df_test_age_categ["age"].var()
y = df_test_age_categ["categ"].var()
F = x/y
df1 = len(str(x)) - 1
df2 = len(str(y)) - 1

In [ ]:
alpha = 0.05
p_value = st.f.cdf(F, df1, df2)
print(p_value)

In [ ]:
if p_value > alpha:
    print("Nous rejetons l'hypothèse H0 var(x) = var(y), il y a ici p_value: {:.2%} de chance que H0 soit rejeté à tord ".format(p) )
else:
    print("var(x) = var(y), on ne peux rejeter l(hypothèse H0, il y a ici p_value: {:.2%} de chance que H0 ne soit pas rejeté à tord ".format(p) )

## Réalisez une régression linéaire simple

In [ ]:
df_test_age_categ.head(2)

In [ ]:
reg_simp = smf.ols('age ~ categ', data=df_test_age_categ).fit()

In [ ]:
print(reg_simp.summary())

## Visualisez la droite de régression

In [ ]:
ax = sns.lmplot(x="age", y="CA",hue="categ", data=df_test_age_categ, ci=None, line_kws={'color':'black'})
ax.set(xlabel='Age', ylabel='CA')
plt.show()

## Test de la normalité des résidus

In [ ]:
df_test_age_categ['residu_s'] = reg_simp.resid

In [ ]:

shapiro(reg_simp.resid)

In [ ]:
shapiro(reg_simp.resid)
p=pvalue
if p < 0.05:
    print("Ici, l'hypothèse de normalité est remise en cause") 
else:
    print("Ici, l'hypothèse de normalité semble vérifiée")   
print(p)

In [ ]:
plt.hist(df_test_age_categ['residu_s'], density=True)
plt.xlabel('Résidus')
plt.title('Histogramme des résidus')
plt.show()

## Anova sans CA

In [ ]:
anova2 = st.f_oneway(df_test_age_categ["age"],df_test_age_categ["categ"])
anova2

In [ ]:
p_value = anova2[1]
# si pvalue inferieur a .05 alors il y a un lien, on rejette donc  H0

if p_value<0.05:
    print("P_value = {}".format(p_value), "La p-valeur de ce test est inférieur à 5%,  l\'hypothèse H0 peux etre rejetée, il existe ici pvalue:, {}".format(p_value), "de chance que H0 ne soit pas rejeté à tord (Beta: Erreur de 2nd espèce)""")
    
else: 
    print("P_value = {}".format(p_value),"La p-valeur est supérieure à 5%, cela signifie qu'il existe pas de lien entre les variables, ce qui incite à rejeter l'hypothèse  H0,  il existe ici p_value:, {}".format(p_value), "de chance que H0 soit rejeté à tord (Alpha: Erreur de 1ère espèce)")

In [ ]:
anova3 = st.f_oneway(df_final["age"],df_final["categ"])
anova3

In [ ]:
p_value = anova3[1]
# si pvalue inferieur a .05 alors il y a pas de lien, on rejette donc pas H0

if p_value<0.05:
    print("P_value = {}".format(p_value), "La p-valeur de ce test est inférieur à 5%,  l\'hypothèse H0 peux etre rejetée, Au moins une des catégories (moyenne) à un lien, il existe ici pvalue:, {}".format(p_value), "de chance que H0 ne soit pas rejeté à tord (Beta: Erreur de 2nd espèce)""")
    
else: 
    print("P_value = {}".format(p_value),"La p-valeur est supérieure à 5%, cela signifie qu'il existe pas de lien entre les variables, ce qui incite à rejeter l'hypothèse  H0,  il existe ici p_value:, {}".format(p_value), "de chance que H0 soit rejeté à tord (Alpha: Erreur de 1ère espèce)")

In [ ]:
categ = df_final.categ.unique()
categ

In [ ]:
categ_ages = {cat: df_final.loc[df_final['categ']==cat, 'age'] for cat in categ}


In [ ]:

plt.boxplot([
    categ_ages[0],
    categ_ages[1],
    categ_ages[2]
],
    labels=['cat. 0', 'cat. 1', 'cat. 2'],
    vert=False, showmeans=True, showfliers=False
)
plt.xlabel('age')
plt.show()

In [ ]:
f, p = st.f_oneway(categ_ages[0], categ_ages[1], categ_ages[2])
print(f, p)

if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0), La p-valeur de ce test est inférieur à 5%')
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%""")  

In [ ]:
eta2(df_final, 'génération', 'categ')

## Age _ panier_moyen_age

In [ ]:
panier_moyen_age

In [ ]:
ax = sns.lmplot(x="age", y="price", data=panier_moyen_age, ci=None, line_kws={'color':'black'})
ax.set(xlabel='Age', ylabel='Panier Moyen')
plt.show()

In [ ]:
géné = df_final.génération.unique()
géné

In [ ]:
panier_moyen_age

In [ ]:
panier_m = {pan: panier_moyen_age.loc[panier_moyen_age['price']==pan, 'age'] for pan in géné}

In [ ]:
panier_m

In [ ]:
plt.boxplot([panier_m[0],
             panier_m[1],
             panier_m[2]],
    labels=['Moins de 30', '30-50', '50 et plus'],
    vert=False, showmeans=True, showfliers=False
)
plt.xlabel('age')
plt.show()

In [ ]:
data1 = panier_moyen_age["age"]
data2 = panier_moyen_age["price"]
coef, p = spearmanr(data1, data2)
print('Coefficient de correlation methode Spearman  : %.3f' % coef, "p_value: {}".format(p))


if p < 0.05:
    print('Les variables sembles etre dépendantes (on rejette H0) p=%.3f' % p, "La p-valeur de ce test est inférieur à 5%,  il existe ici p_value:, {:.2%}"""
      .format(p), "de chance que H0 soit rejeté à tord (Alpha: Erreur de 1ère espèce)""") 
else:
    print("""les variables semblent indépendantes, La p-valeur est supérieure à 5%, l\'hypothèse H0 ne peux etre rejetée, il existe ici pvalue:, {:.2%}"""
          .format(p), "de chance que H0 ne soit pas rejeté à tord (Beta: Erreur de 2nd espèce)""")   


In [ ]:
t = pd.crosstab(df_final.génération, "freq")
t.plot.pie(subplots=True, figsize = (6, 6))
plt.title("Masse des générations");

In [ ]:
df_final.columns

In [ ]:
t = pd.crosstab(df_final.date_année, "freq")
t.plot.pie(subplots=True, figsize = (6, 6))
plt.title("Session par année");

In [ ]:
df_test_Age

In [ ]:
panier_moyen_age

In [ ]:
sns.set()
#sns.set(rc={'figure.figsize': (11.7,8.27)}) 

ax = sns.boxplot(x="price", y="génération", data=df_test_Age)
plt.xlabel('Panier_moyen',size=20)
plt.ylabel('Génération',size=20)
plt.title('Boîtes à moustaches, Panier par génération')

plt.show()


# Probabilité qu’un client achète la référence 0_525 sachant qu’il a acheté la référence 2_159.

In [ ]:
df_final.columns

In [ ]:
# P(A ∩ B) = P(A)P(B)

## A = vente_0_525

In [ ]:
# Client ayant acheté la référence 0_525
vente_0_525 = df_final.loc[df_final["id_prod"]=="0_525"]
#set2.set_index(["client_id"], inplace=True)
vente_0_525

In [ ]:
vente_0_525.shape

In [ ]:
A=vente_0_525.groupby("client_id").agg({"price":"sum"})

In [ ]:
# client_id achat multiple
vente_0_525["client_id"].duplicated().sum()

In [ ]:
len(A)

## B = vente_2_159

In [ ]:
# Client ayant acheté la référence 2_159
vente_2_159 = df_final.loc[df_final["id_prod"]=="2_159"]
#set1.set_index(["client_id"], inplace=True)
vente_2_159

In [ ]:
# client_id achat multiple
vente_2_159["client_id"].duplicated().sum()

In [ ]:
B=vente_2_159.groupby("client_id").agg({"price":"sum"})

In [ ]:
len(B)

## Calcul de la probabilité

In [ ]:
n=len(df_final.client_id.unique())

In [ ]:
A=len(vente_0_525.client_id.unique())

In [ ]:
B=len(vente_2_159.client_id.unique())

In [ ]:
PROBA_B= B/n
PROBA_A= A/n
PAinterB = PROBA_A/PROBA_B
print(PAinterB)

In [ ]:
print(" La probabilité qu’un client achète la référence 0_525 sachant qu’il a acheté la référence 2_159 est de: {:.2%}".format(PAinterB))